In [1]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

#from datasets import load_datasets_and_vocabs, my_collate
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
from tree import *

import warnings
warnings.filterwarnings('always')

In [2]:
def set_seed(args):
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    torch.cuda.manual_seed_all(args.seed)

In [3]:
from torch.nn.utils.rnn import pad_sequence
import argparse
import codecs
import json
import linecache
import logging
import os
import pickle
import random
import sys
from collections import Counter, defaultdict
from copy import copy, deepcopy

import nltk
import numpy as np
import simplejson as json
import torch
from allennlp.modules.elmo import batch_to_ids
from lxml import etree
from nltk import word_tokenize
from nltk.tokenize import TreebankWordTokenizer
from torch.utils.data import DataLoader, Dataset



def load_datasets_and_vocabs(args):
    train, test = get_dataset(args.dataset_name)

    # Our model takes unrolled data, currently we don't consider the MAMS cases(future experiments)
    train_all_unrolled = get_rolled_and_unrolled_data(train, args)
    test_all_unrolled = get_rolled_and_unrolled_data(test, args)

    print("Size of the train dataset: {}".format(len(train_all_unrolled)))
    print("Size of the test dataset: {}".format(len(test_all_unrolled)))
    print("Printing all unrolled")
    print(train_all_unrolled[:2])

    # Build word vocabulary(part of speech, dep_tag) and save pickles.
    word_vecs, word_vocab, dep_tag_vocab, pos_tag_vocab = load_and_cache_vocabs(train_all_unrolled+test_all_unrolled, args)

    embedding = torch.from_numpy(np.asarray(word_vecs, dtype=np.float32))
    args.glove_embedding = embedding

    train_dataset = ASBA_Depparsed_Dataset(train_all_unrolled, args, word_vocab, dep_tag_vocab, pos_tag_vocab)
    
    test_dataset = ASBA_Depparsed_Dataset(test_all_unrolled, args, word_vocab, dep_tag_vocab, pos_tag_vocab)

    return train_dataset, test_dataset, word_vocab, dep_tag_vocab, pos_tag_vocab


def read_sentence_depparsed(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
        return data


def get_dataset(dataset_name):
    '''
    Already preprocess the data and now they are in json format.(only for semeval14)
    Retrieve train and test set
    With a list of dict:
    e.g. {"sentence": "Boot time is super fast, around anywhere from 35 seconds to 1 minute.",
    "tokens": ["Boot", "time", "is", "super", "fast", ",", "around", "anywhere", "from", "35", "seconds", "to", "1", "minute", "."],
    "tags": ["NNP", "NN", "VBZ", "RB", "RB", ",", "RB", "RB", "IN", "CD", "NNS", "IN", "CD", "NN", "."],
    "predicted_dependencies": ["nn", "nsubj", "root", "advmod", "advmod", "punct", "advmod", "advmod", "prep", "num", "pobj", "prep", "num", "pobj", "punct"],
    "predicted_heads": [2, 3, 0, 5, 3, 5, 8, 5, 8, 11, 9, 9, 14, 12, 3],
    "dependencies": [["nn", 2, 1], ["nsubj", 3, 2], ["root", 0, 3], ["advmod", 5, 4], ["advmod", 3, 5], ["punct", 5, 6], ["advmod", 8, 7], ["advmod", 5, 8],
                    ["prep", 8, 9], ["num", 11, 10], ["pobj", 9, 11], ["prep", 9, 12], ["num", 14, 13], ["pobj", 12, 14], ["punct", 3, 15]],
    "aspect_sentiment": [["Boot time", "positive"]], "from_to": [[0, 2]]}
    '''
    rest_train = 'data/semeval14/Restaurants_Train_v2_biaffine_depparsed_with_energy.json'
    rest_test = 'data/semeval14/Restaurants_Test_Gold_biaffine_depparsed_with_energy.json'

    laptop_train = 'data/semeval14/Laptop_Train_v2_biaffine_depparsed.json'
    laptop_test = 'data/semeval14/Laptops_Test_Gold_biaffine_depparsed.json'

    twitter_train = 'data/twitter/train_biaffine.json'
    twitter_test = 'data/twitter/test_biaffine.json'
    print("Using the dataset: {}".format(dataset_name))
    ds_train = {'rest': rest_train,
                'laptop': laptop_train, 
                'twitter': twitter_train}
    ds_test = {'rest': rest_test,
               'laptop': laptop_test, 
               'twitter': twitter_test}

    train = list(read_sentence_depparsed(ds_train[dataset_name]))
    test = list(read_sentence_depparsed(ds_test[dataset_name]))
    
    print("Read {} Train set: {}".format(dataset_name, len(train)))
    print("Read {} Test set: {}".format(dataset_name, len(test)))

    return train, test


def reshape_dependency_tree(as_start, as_end, dependencies, tokens=None, max_hop = 5):
    '''
    Adding multi hops
    This function is at the core of our algo, it reshape the dependency tree and center on the aspect.
    In open-sourced edition, I choose not to take energy(the soft prediction of dependency from parser)
    into consideration. For it requires tweaking allennlp's source code, and the energy is space-consuming.
    And there are no significant difference in performance between the soft and the hard(with non-connect) version.

    '''
    dep_tag = []
    dep_idx = []
    dep_dir = []
    # 1 hop

    for i in range(as_start, as_end):
        for dep in dependencies:
            if i == dep[1] - 1:
                # not root, not aspect
                if (dep[2] - 1 < as_start or dep[2] - 1 >= as_end) and dep[2] != 0 and dep[2] - 1 not in dep_idx:
                    if str(dep[0]) != 'punct':  # and tokens[dep[2] - 1] not in stopWords
                        dep_tag.append(dep[0])
                        dep_dir.append(1)
                    else:
                        dep_tag.append('<pad>')
                        dep_dir.append(0)
                    dep_idx.append(dep[2] - 1)
            elif i == dep[2] - 1:
                # not root, not aspect
                if (dep[1] - 1 < as_start or dep[1] - 1 >= as_end) and dep[1] != 0 and dep[1] - 1 not in dep_idx:
                    if str(dep[0]) != 'punct':  # and tokens[dep[1] - 1] not in stopWords
                        dep_tag.append(dep[0])
                        dep_dir.append(2)
                    else:
                        dep_tag.append('<pad>')
                        dep_dir.append(0)
                    dep_idx.append(dep[1] - 1)

    # add aspect and index, to make sure length matches len(tokens)
    for idx, token in enumerate(tokens):
        if idx not in dep_idx:
            dep_tag.append('<pad>')
            dep_dir.append(0)
            dep_idx.append(idx)

    index = [i[0] for i in sorted(enumerate(dep_idx), key=lambda x:x[1])]
    dep_tag = [dep_tag[i] for i in index]
    dep_idx = [dep_idx[i] for i in index]
    dep_dir = [dep_dir[i] for i in index]

    assert len(tokens) == len(dep_idx), 'length wrong'
    return dep_tag, dep_idx, dep_dir




def get_rolled_and_unrolled_data(input_data, args):
    '''
    In input_data, each sentence could have multiple aspects with different sentiments.
    Our method treats each sentence with one aspect at a time, so even for
    multi-aspect-multi-sentiment sentences, we will unroll them to single aspect sentence.

    Perform reshape_dependency_tree to each sentence with aspect

    return:
        all_rolled:
                a list of dict
                    {sentence, tokens, pos_tags, pos_class, aspects(list of aspects), sentiments(list of sentiments)
                    froms, tos, dep_tags, dep_index, dependencies}
        all_unrolled:
                unrolled, with aspect(single), sentiment(single) and so on...
        mixed_rolled:
                Multiple aspects and multiple sentiments, ROLLED.
        mixed_unrolled:
                Multiple aspects and multiple sentiments, UNROLLED.
    '''
    # A hand-picked set of part of speech tags that we see contributes to ABSA.
    opinionated_tags = ['JJ', 'JJR', 'JJS', 'RB', 'RBR',
                        'RBS', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']

    all_rolled = []
    all_unrolled = []
    mixed_rolled = []
    mixed_unrolled = []

    unrolled = []
    mixed = []
    unrolled_ours = []
    mixed_ours = []

    # Make sure the tree is successfully built.
    zero_dep_counter = 0

    # Sentiment counters
    total_counter = defaultdict(int)
    mixed_counter = defaultdict(int)
    sentiments_lookup = {'negative': 0, 'positive': 1, 'neutral': 2}

    tree_samples = []
    # for seeking 'but' examples
    for e in input_data:
        e['tokens'] = [x.lower() for x in e['tokens']]
        aspects = []
        sentiments = []
        froms = []
        tos = []
        dep_tags = []
        dep_index = []
        dep_dirs = []

        # Classify based on POS-tags

        pos_class = e['tags']

        # Iterate through aspects in a sentence and reshape the dependency tree.
        for i in range(len(e['aspect_sentiment'])):
            aspect = e['aspect_sentiment'][i][0].lower()
            # We would tokenize the aspect while at it.
            aspect = word_tokenize(aspect)
            sentiment = sentiments_lookup[e['aspect_sentiment'][i][1]]
            frm = e['from_to'][i][0]
            to = e['from_to'][i][1]

            aspects.append(aspect)
            sentiments.append(sentiment)
            froms.append(frm)
            tos.append(to)

            # Center on the aspect.
            dep_tag, dep_idx, dep_dir = reshape_dependency_tree(frm, to, e['dependencies'], tokens=e['tokens'], max_hop=args.max_hop)

            # Because of tokenizer differences, aspect opsitions are off, so we find the index and try again.
            if len(dep_tag) == 0:
                zero_dep_counter += 1
                as_sent = e['aspect_sentiment'][i][0].split()
                as_start = e['tokens'].index(as_sent[0])
                # print(e['tokens'], e['aspect_sentiment'], e['dependencies'],as_sent[0])
                as_end = e['tokens'].index(
                    as_sent[-1]) if len(as_sent) > 1 else as_start + 1
                print("Debugging: as_start as_end ", as_start, as_end)
                dep_tag, dep_idx, dep_dir = reshape_dependency_tree(as_start, as_end, e['dependencies'], tokens=e['tokens'], max_hop=args.max_hop)
                if len(dep_tag) == 0:  # for debugging
                    print("Debugging: zero_dep",
                          e['aspect_sentiment'][i][0], e['tokens'])
                    print("Debugging: ". e['dependencies'])
                else:
                    zero_dep_counter -= 1

            dep_tags.append(dep_tag)
            dep_index.append(dep_idx)
            dep_dirs.append(dep_dir)

            total_counter[e['aspect_sentiment'][i][1]] += 1

            # Unrolling
            all_unrolled.append(
                {'sentence': e['tokens'], 'tags': e['tags'], 'pos_class': pos_class, 'aspect': aspect, 'sentiment': sentiment,
                    'predicted_dependencies': e['predicted_dependencies'], 'predicted_heads': e['predicted_heads'],
                 'from': frm, 'to': to, 'dep_tag': dep_tag, 'dep_idx': dep_idx, 'dep_dir':dep_dir,'dependencies': e['dependencies']})




    return all_unrolled

import pickle

def load_and_cache_vocabs(data, args):
    '''
    Build vocabulary of words, part of speech tags, dependency tags and cache them.
    Load glove embedding if needed.'''
    
    if True:
        with open("cached_word_vocab.pkl", 'rb') as f:
            word_vocab = pickle.load(f)
        
        with open("cached_word_vecs.pkl", 'rb') as f:
            word_vecs = pickle.load(f)
        
        with open("cached_dep_tag_vocab.pkl", 'rb') as f:
            dep_tag_vocab = pickle.load(f)
            
        with open("cached_pos_tag_vocab.pkl", 'rb') as f:
            pos_tag_vocab = pickle.load(f)
        #pos_tag_vocab = build_pos_tag_vocab(data, min_freq=0)
        
    
    else:
        
        # Build or load word vocab and glove embeddings.
        word_vocab = build_text_vocab(data)
        print('Word vocab size: {}'.format(word_vocab['len']))
        with open("cached_word_vocab.pkl", 'wb') as f:
            pickle.dump(word_vocab, f, -1)
        
        word_vecs = load_glove_embedding(word_vocab['itos'], args.glove_dir, 0.25, args.embedding_dim)
        print('Word vecs size')
        with open("cached_word_vecs.pkl", 'wb') as f:
            pickle.dump(word_vecs, f, -1)
            
        # Build vocab of dependency tags
        dep_tag_vocab = build_dep_tag_vocab(data, min_freq=0)
        print('dep_tag_vocab size: {}'.format(dep_tag_vocab['len']))
        with open("cached_dep_tag_vocab.pkl", 'wb') as f:
            pickle.dump(dep_tag_vocab, f, -1)
            
        # Build vocab of part of speech tags.
        pos_tag_vocab = build_pos_tag_vocab(data, min_freq=0)
        print('pos_tag_vocab size: {}'.format(pos_tag_vocab['len']))
        with open("cached_pos_tag_vocab.pkl", 'wb') as f:
            pickle.dump(pos_tag_vocab, f, -1)

    return word_vecs, word_vocab, dep_tag_vocab, pos_tag_vocab


def load_glove_embedding(word_list, glove_dir, uniform_scale, dimension_size):
    glove_words = []
    with open(os.path.join(glove_dir, 'glove.840B.300d.txt'), 'r', encoding="utf8") as fopen:
        for line in fopen:
            glove_words.append(line.strip().split(' ')[0])
    word2offset = {w: i for i, w in enumerate(glove_words)}
    word_vectors = []
    for word in word_list:
        if word in word2offset:
            line = linecache.getline(os.path.join(
                glove_dir, 'glove.840B.300d.txt'), word2offset[word]+1)
            assert(word == line[:line.find(' ')].strip())
            word_vectors.append(np.fromstring(
                line[line.find(' '):].strip(), sep=' ', dtype=np.float32))
        elif word == '<pad>':
            word_vectors.append(np.zeros(dimension_size, dtype=np.float32))
        else:
            word_vectors.append(
                np.random.uniform(-uniform_scale, uniform_scale, dimension_size))
    return word_vectors


def _default_unk_index():
    return 1


def build_text_vocab(data, vocab_size=100000, min_freq=2):
    counter = Counter()
    for d in data:
        s = d['sentence']
        counter.update(s)

    itos = ['[PAD]', '[UNK]']
    min_freq = max(min_freq, 1)

    # sort by frequency, then alphabetically
    words_and_frequencies = sorted(counter.items(), key=lambda tup: tup[0])
    words_and_frequencies.sort(key=lambda tup: tup[1], reverse=True)

    for word, freq in words_and_frequencies:
        if freq < min_freq or len(itos) == vocab_size:
            break
        itos.append(word)
    # stoi is simply a reverse dict for itos
    stoi = defaultdict(_default_unk_index)
    stoi.update({tok: i for i, tok in enumerate(itos)})

    return {'itos': itos, 'stoi': stoi, 'len': len(itos)}


def build_pos_tag_vocab(data, vocab_size=1000, min_freq=1):
    """
    Part of speech tags vocab.
    """
    counter = Counter()
    for d in data:
        tags = d['tags']
        counter.update(tags)

    itos = ['<pad>']
    min_freq = max(min_freq, 1)

    # sort by frequency, then alphabetically
    words_and_frequencies = sorted(counter.items(), key=lambda tup: tup[0])
    words_and_frequencies.sort(key=lambda tup: tup[1], reverse=True)

    for word, freq in words_and_frequencies:
        if freq < min_freq or len(itos) == vocab_size:
            break
        itos.append(word)
    # stoi is simply a reverse dict for itos
    stoi = defaultdict()
    stoi.update({tok: i for i, tok in enumerate(itos)})

    return {'itos': itos, 'stoi': stoi, 'len': len(itos)}



def build_dep_tag_vocab(data, vocab_size=1000, min_freq=0):
    counter = Counter()
    for d in data:
        tags = d['dep_tag']
        counter.update(tags)

    itos = ['<pad>', '<unk>']
    min_freq = max(min_freq, 1)

    # sort by frequency, then alphabetically
    words_and_frequencies = sorted(counter.items(), key=lambda tup: tup[0])
    words_and_frequencies.sort(key=lambda tup: tup[1], reverse=True)

    for word, freq in words_and_frequencies:
        if freq < min_freq or len(itos) == vocab_size:
            break
        if word == '<pad>':
            continue
        itos.append(word)
    # stoi is simply a reverse dict for itos
    stoi = defaultdict(_default_unk_index)
    stoi.update({tok: i for i, tok in enumerate(itos)})

    return {'itos': itos, 'stoi': stoi, 'len': len(itos)}


class ASBA_Depparsed_Dataset(Dataset):
    def __init__(self, data, args, word_vocab, dep_tag_vocab, pos_tag_vocab):
        self.data = data
        self.args = args
        self.word_vocab = word_vocab
        self.dep_tag_vocab = dep_tag_vocab
        self.pos_tag_vocab = pos_tag_vocab

        self.convert_features()

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        e = self.data[idx]
        items = e['dep_tag_ids'], e['pos_class'], e['text_len'], e['aspect_len'], e['sentiment'], e['dep_rel_ids'], e['predicted_heads'], e['aspect_position'], e['dep_dir_ids']
        non_bert_items = e['sentence_ids'], e['aspect_ids']
        items_tensor = non_bert_items + items
        items_tensor = tuple(torch.tensor(t) for t in items_tensor)

        return items_tensor

    

    def convert_features(self):
        '''
        Convert sentence, aspects, pos_tags, dependency_tags to ids.
        '''
        for i in range(len(self.data)):
       
            self.data[i]['sentence_ids'] = [self.word_vocab['stoi'][w] for w in self.data[i]['sentence']]
            self.data[i]['aspect_ids'] = [self.word_vocab['stoi'][w] for w in self.data[i]['aspect']]
           

            self.data[i]['text_len'] = len(self.data[i]['sentence'])
            self.data[i]['aspect_position'] = [0] * self.data[i]['text_len']
            try:  # find the index of aspect in sentence
                for j in range(self.data[i]['from'], self.data[i]['to']):
                    self.data[i]['aspect_position'][j] = 1
            except:
                for term in self.data[i]['aspect']:
                    self.data[i]['aspect_position'][self.data[i]
                                                    ['sentence'].index(term)] = 1

            self.data[i]['dep_tag_ids'] = [self.dep_tag_vocab['stoi'][w]
                                           for w in self.data[i]['dep_tag']]
            self.data[i]['dep_dir_ids'] = [idx
                                           for idx in self.data[i]['dep_dir']]
            self.data[i]['pos_class'] = [self.pos_tag_vocab['stoi'][w]
                                             for w in self.data[i]['tags']]
            self.data[i]['aspect_len'] = len(self.data[i]['aspect'])

            self.data[i]['dep_rel_ids'] = [self.dep_tag_vocab['stoi'][r]
                                           for r in self.data[i]['predicted_dependencies']]


def my_collate(batch):
    '''
    Pad sentence and aspect in a batch.
    Sort the sentences based on length.
    Turn all into tensors.
    '''
    sentence_ids, aspect_ids, dep_tag_ids, pos_class, text_len, aspect_len, sentiment, dep_rel_ids, dep_heads, aspect_positions, dep_dir_ids = zip(
        *batch)  # from Dataset.__getitem__()
    text_len = torch.tensor(text_len)
    aspect_len = torch.tensor(aspect_len)
    sentiment = torch.tensor(sentiment)

    # Pad sequences.
    sentence_ids = pad_sequence(
        sentence_ids, batch_first=True, padding_value=0)
    aspect_ids = pad_sequence(aspect_ids, batch_first=True, padding_value=0)
    aspect_positions = pad_sequence(
        aspect_positions, batch_first=True, padding_value=0)

    dep_tag_ids = pad_sequence(dep_tag_ids, batch_first=True, padding_value=0)
    dep_dir_ids = pad_sequence(dep_dir_ids, batch_first=True, padding_value=0)
    pos_class = pad_sequence(pos_class, batch_first=True, padding_value=0)

    dep_rel_ids = pad_sequence(dep_rel_ids, batch_first=True, padding_value=0)
    dep_heads = pad_sequence(dep_heads, batch_first=True, padding_value=0)

    # Sort all tensors based on text len.
    _, sorted_idx = text_len.sort(descending=True)
    sentence_ids = sentence_ids[sorted_idx]
    aspect_ids = aspect_ids[sorted_idx]
    aspect_positions = aspect_positions[sorted_idx]
    dep_tag_ids = dep_tag_ids[sorted_idx]
    dep_dir_ids = dep_dir_ids[sorted_idx]
    pos_class = pos_class[sorted_idx]
    text_len = text_len[sorted_idx]
    aspect_len = aspect_len[sorted_idx]
    sentiment = sentiment[sorted_idx]
    dep_rel_ids = dep_rel_ids[sorted_idx]
    dep_heads = dep_heads[sorted_idx]

    return sentence_ids, aspect_ids, dep_tag_ids, pos_class, text_len, aspect_len, sentiment, dep_rel_ids, dep_heads, aspect_positions, dep_dir_ids

D:\anaconda\lib\site-packages\google\api_core\exceptions.py:37: ImportWarning: Please install grpcio-status to obtain helpful grpc error messages.
  warnings.warn(
D:\anaconda\lib\site-packages\botocore\httpsession.py:41: DeprecationWarning: 'urllib3.contrib.pyopenssl' module is deprecated and will be removed in a future release of urllib3 2.x. Read more in this issue: https://github.com/urllib3/urllib3/issues/2680
  from urllib3.contrib.pyopenssl import orig_util_SSLContext as SSLContext
D:\anaconda\lib\site-packages\torchvision\transforms\functional_pil.py:228: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  interpolation: int = Image.BILINEAR,
D:\anaconda\lib\site-packages\torchvision\transforms\functional_pil.py:295: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  interpolation: int = Image.NEAREST,
D:\anaconda\lib\si

In [4]:
def mask_logits(target, mask):
    return target * mask + (1 - mask) * (-1e30)

class Gating(nn.Module):
    def __init__(self, layer_num, dim):
        super().__init__()
        self.layer_num = layer_num
        self.linear = nn.ModuleList([nn.Linear(dim, dim) for _ in range(layer_num)])
        self.gate = nn.ModuleList([nn.Linear(dim, dim) for _ in range(layer_num)])

    def forward(self, x):
        for i in range(self.layer_num):
            gate = torch.sigmoid(self.gate[i](x))
            nonlinear = F.relu(self.linear[i](x))
            x = gate * nonlinear + (1 - gate) * x
        return x

class DotprodAttention(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, feature, dep_tags, dmask):
        '''
        C feature/context [N, L, D]
        Q dep_tags_v          [N, D]
        mask dmask          [N, L]
        '''

        Q = dep_tags
        Q = Q.unsqueeze(2)  # (N, D, 1)
        dot_prod = torch.bmm(feature, Q)  # (N, L, 1)
        dmask = dmask.unsqueeze(2)  # (N, D, 1)
        attention_weight = mask_logits(dot_prod, dmask)  # (N, L ,1)
        attention = F.softmax(attention_weight, dim=1)  # (N, L, 1)

        out = torch.bmm(feature.transpose(1, 2), attention)  # (N, D, 1)
        out = out.squeeze(2)
        out = torch.sigmoid(out)
        # (N, D), ([N, L]), (N, L, 1)
        return out

class RelationAttention(nn.Module):
    def __init__(self, in_dim = 300, hidden_dim = 64):
        # in_dim: the dimension fo query vector
        super().__init__()

        self.fc1 = nn.Linear(in_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, 1)

    def forward(self, feature, dep_tags_v, dmask):
        '''
        C feature/context [N, L, D]
        Q dep_tags_v          [N, L, D]
        mask dmask          [N, L]
        '''
        Q = self.fc1(dep_tags_v)
        Q = self.relu(Q)
        Q = self.fc2(Q)  # (N, L, 1)
        Q = Q.squeeze(2)
        Q = F.softmax(mask_logits(Q, dmask), dim=1)

        Q = Q.unsqueeze(2)
        out = torch.bmm(feature.transpose(1, 2), Q)
        out = out.squeeze(2)
        # out = F.sigmoid(out)
        return out  # ([N, L])

In [5]:
class TextGraphAttentionNetwork(nn.Module):
    def __init__(self, args, dep_tag_num):
        super(TextGraphAttentionNetwork, self).__init__()
        self.args = args

        num_embeddings, embed_dim = args.glove_embedding.shape
        self.embed = nn.Embedding(num_embeddings, embed_dim)
        self.embed.weight = nn.Parameter(args.glove_embedding, requires_grad=False)
        self.dropout = nn.Dropout(args.dropout)
        self.tanh = nn.Tanh()
        self.gate = Gating(args.num_layers, args.embedding_dim)
        self.rnn = nn.GRU(input_size=args.embedding_dim, hidden_size=args.hidden_size, bidirectional=True, batch_first=True, num_layers=args.num_layers)
    
        self.gat = [DotprodAttention().to(args.device) for i in range(args.num_heads)]
        self.gat_dep = [RelationAttention(in_dim = args.embedding_dim).to(args.device) for i in range(args.num_heads)]
        
        last_hidden_size = args.hidden_size*4

        layers = [nn.Linear(last_hidden_size, args.final_hidden_size), nn.ReLU()]
        
        for _ in range(args.num_mlps-1):
            layers += [nn.Linear(args.final_hidden_size, args.final_hidden_size), nn.ReLU()]
            
        self.dep_embed = nn.Embedding(dep_tag_num, args.embedding_dim)
        gcn_input_dim = args.hidden_size * 2
        self.fc = nn.Linear(args.embedding_dim, 2*args.hidden_size)
        self.fcs = nn.Sequential(*layers)
        self.fc_final = nn.Linear(args.final_hidden_size, args.num_classes)
        self.linear = nn.Linear(gcn_input_dim, gcn_input_dim)

    def forward(self, sentence, aspect, pos_class, dep_tags, text_len, aspect_len, dep_rels, dep_heads, aspect_position, dep_dirs):

        '''
        Forward takes:
            sentence: sentence_id of size (batch_size, text_length)
            aspect: aspect_id of size (batch_size, aspect_length)
            pos_class: pos_tag_id of size (batch_size, text_length)
            dep_tags: dep_tag_id of size (batch_size, text_length)
            text_len: (batch_size,) length of each sentence
            aspect_len: (batch_size, ) aspect length of each sentence
            dep_rels: (batch_size, text_length) relation
            dep_heads: (batch_size, text_length) which node adjacent to that node
            aspect_position: (batch_size, text_length) mask, with the position of aspect as 1 and others as 0
            dep_dirs: (batch_size, text_length) the directions each node to the aspect
        '''
        fmask = (torch.zeros_like(sentence) != sentence).float()  # (N，L)
        feature = self.embed(sentence)  # (N, L, D)
        feature = self.dropout(feature)
        feature = self.gate(feature)
        feature = self.fc(feature) # (N,L,D)
        
        dep_feature = self.dep_embed(dep_tags) 
        dep_feature = self.gate(dep_feature)

        dep_out = [g(feature, dep_feature, fmask).unsqueeze(1) for g in self.gat_dep] #(N, 1, D) * num_heads
        dep_out = torch.cat(dep_out, dim = 1) # (N, H, D)
        dep_out = dep_out.mean(dim = 1) # (N, D)
        
        #print("Shape of the feature", feature.shape)
        #print("Shape of the dep_out", dep_out.shape)
        
        gat_out = self.linear(feature) # (N, L, D)
        #print("Shape of the gat_out right now", gat_out.shape)
        fmask = fmask.unsqueeze(2)
        gat_out = gat_out * fmask
        
        #print("Shape of the gat_out after fmask", gat_out.shape)
        gat_out = F.relu(torch.sum(gat_out, dim = 1)) # (N, D)
        #print("Shape of the gat_out", gat_out.shape)
        
        feature_out = torch.cat([dep_out,  gat_out], dim = 1) # (N, D')
       
        # feature_out = gat_out
        #############################################################################################
        x = self.dropout(feature_out)
        x = self.fcs(x)
        logit = self.fc_final(x)
        return logit

In [6]:
class Parameters:
    def __init__(self):
        pass 

args = Parameters()
 # Required parameters
args.dataset_name = 'laptop' # 'rest' 'laptop' 'twitter'
args.output_dir = 'data/output-gcn'
args.num_classes = 3
args.cuda_id = '3'
args.seed = 2022

# Model parameters
args.glove_dir = 'glove'
args.num_layers = 2
args.max_hop = 4
args.num_heads = 6
args.dropout = 0
args.num_gcn_layers = 1
args.gc_mem_dim = 300

args.gcn_dropout = 0.2
# GAT
args.gat = True
args.gat_out = True


args.gat_attention_type = 'dotprod' # 'linear' 'dotprod' 'gcn'
args.embedding_type = 'glove' # 'glove' 'bert'
args.embedding_dim = 300
args.dep_relation_embed_dim = 300
args.hidden_size = 300
args.final_hidden_size = 300
args.num_mlps = 2
# Training parameters
args.per_gpu_train_batch_size = 16
args.per_gpu_eval_batch_size = 32
args.gradient_accumulation_steps = 2
args.learning_rate = 1e-3
args.weight_decay = 0.0
args.adam_epsilon = 1e-8
args.max_grad_norm = 1.0
args.num_train_epochs = 30
args.max_steps = -1
args.logging_steps = 50

In [7]:
def get_input_from_batch(args, batch):
    # sentence_ids, aspect_ids, dep_tag_ids, pos_class, text_len, aspect_len, sentiment, dep_rel_ids, dep_heads, aspect_positions
    inputs = {  'sentence': batch[0],
                'aspect': batch[1], # aspect token
                'dep_tags': batch[2], # reshaped
                'pos_class': batch[3],
                'text_len': batch[4],
                'aspect_len': batch[5],
                'dep_rels': batch[7], # adj no-reshape
                'dep_heads': batch[8],
                'aspect_position': batch[9],
                'dep_dirs': batch[10]
                }
    labels = batch[6]

    return inputs, labels

def train(args, train_dataset, model, eval_dataset):
    '''Train the model'''
    args.train_batch_size = args.per_gpu_train_batch_size
    args.eval_batch_size = args.per_gpu_eval_batch_size

    train_sampler = RandomSampler(train_dataset)
    eval_sampler = SequentialSampler(eval_dataset)

    train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=args.train_batch_size, collate_fn=my_collate)
    eval_dataloader = DataLoader(eval_dataset, sampler=eval_sampler, batch_size=args.eval_batch_size, collate_fn=my_collate)


    parameters = filter(lambda param: param.requires_grad, model.parameters())
    optimizer = torch.optim.Adam(parameters, lr=args.learning_rate)

    # Train
    print("Started Training...")
    print("Num examples = {}".format(len(train_dataset)))
    print("Num Epochs = {}".format(args.num_train_epochs))
    print("Instantaneous batch size per GPU = {}".format(args.per_gpu_train_batch_size))

    all_eval_results = []

    set_seed(args)

    for epoch in range(args.num_train_epochs):
        model.train()
        train_loss = 0.0

        for step, batch in enumerate(train_dataloader):
            model.zero_grad()
            batch = tuple(t.to(args.device) for t in batch)
            inputs, labels = get_input_from_batch(args, batch)
            logit = model(**inputs)
            loss = F.cross_entropy(logit, labels)
            loss.backward()
            train_loss += loss.item()
            optimizer.step()

        model.eval()
        eval_loss = 0.0
        preds = None
        out_label_ids = None
        for step, batch in enumerate(eval_dataloader):
            with torch.no_grad():
                batch = tuple(t.to(args.device) for t in batch)
                inputs, labels = get_input_from_batch(args, batch)
                logits = model(**inputs)
                loss = F.cross_entropy(logits, labels)
                eval_loss += loss.item()

                if preds is None:
                    preds = logits.detach().cpu().numpy()
                    out_label_ids = labels.detach().cpu().numpy()
                else:
                    preds = np.append(preds, logits.detach().cpu().numpy(), axis=0)
                    out_label_ids = np.append(out_label_ids, labels.detach().cpu().numpy(), axis=0)

   
        preds = np.argmax(preds, axis=1)
        result = compute_metrics(preds, out_label_ids)
    
        result['train_loss'] = train_loss/len(train_dataloader)
        result['eval_loss'] = eval_loss/len(eval_dataloader)
        result['epoch'] = epoch

        all_eval_results.append(result)
        print("Epoch: {}; train_loss: {}; eval_loss: {}; eval_acc: {}; eval_f1: {}".format(epoch + 1, np.round(train_loss/len(train_dataloader), 4), np.round(eval_loss/len(eval_dataloader), 4), np.round(result['acc'], 4), np.round(result['f1'], 4)))
    
    return all_eval_results

def compute_metrics(preds, labels):
    acc = accuracy_score(y_true=labels, y_pred=preds)
    f1 = f1_score(y_true=labels, y_pred=preds, average='macro')
    precision = precision_score(y_true=labels, y_pred=preds, average='macro')
    recall = recall_score(y_true=labels, y_pred=preds, average='macro')
    return {
        "acc": acc,
        "f1": f1,
        "precision": precision,
        "recall": recall
    }


In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
args.device = device
print('Device is {}'.format(args.device))

# Set seed
set_seed(args)
# Load datasets and vocabs
train_dataset, test_dataset, word_vocab, dep_tag_vocab, pos_tag_vocab= load_datasets_and_vocabs(args)
args.dep_tag_vocab_size = len(dep_tag_vocab['stoi'])
dep_tag_vocab

Device is cuda
Using the dataset: laptop
Read laptop Train set: 1462
Read laptop Test set: 411
Size of the train dataset: 2313
Size of the test dataset: 638
Printing all unrolled
[{'sentence': ['i', 'charge', 'it', 'at', 'night', 'and', 'skip', 'taking', 'the', 'cord', 'with', 'me', 'because', 'of', 'the', 'good', 'battery', 'life', '.'], 'tags': ['PRON', 'VERB', 'PRON', 'ADP', 'NOUN', 'CCONJ', 'VERB', 'VERB', 'DET', 'NOUN', 'ADP', 'PRON', 'SCONJ', 'ADP', 'DET', 'ADJ', 'NOUN', 'NOUN', 'PUNCT'], 'pos_class': ['PRON', 'VERB', 'PRON', 'ADP', 'NOUN', 'CCONJ', 'VERB', 'VERB', 'DET', 'NOUN', 'ADP', 'PRON', 'SCONJ', 'ADP', 'DET', 'ADJ', 'NOUN', 'NOUN', 'PUNCT'], 'aspect': ['cord'], 'sentiment': 2, 'predicted_dependencies': ['nsubj', 'root', 'dep', 'advmod', 'dep', 'nsubj', 'dep', 'dep', 'det', 'dobj', 'prep', 'dep', 'advmod', 'prep', 'dep', 'amod', 'nn', 'pobj', 'punct'], 'predicted_heads': [2, 0, 2, 2, 4, 7, 2, 7, 10, 8, 8, 11, 14, 12, 17, 17, 18, 14, 2], 'from': 9, 'to': 10, 'dep_tag': ['<p

{'itos': ['<pad>',
  '<unk>',
  'dep',
  'det',
  'amod',
  'prep',
  'advmod',
  'nsubj',
  'pobj',
  'nn',
  'ccomp',
  'mark',
  'dobj',
  'cc',
  'cop',
  'num',
  'xcomp',
  'advcl',
  'aux',
  'rcmod',
  'csubj',
  'discourse',
  'neg',
  'poss',
  'conj',
  'pcomp',
  'tmod',
  'quantmod',
  'possessive',
  'number',
  'prt',
  'acomp',
  'parataxis',
  'npadvmod',
  'partmod',
  'auxpass',
  'expl',
  'nsubjpass',
  'infmod',
  'iobj'],
 'stoi': defaultdict(<function __main__._default_unk_index()>,
             {'<pad>': 0,
              '<unk>': 1,
              'dep': 2,
              'det': 3,
              'amod': 4,
              'prep': 5,
              'advmod': 6,
              'nsubj': 7,
              'pobj': 8,
              'nn': 9,
              'ccomp': 10,
              'mark': 11,
              'dobj': 12,
              'cc': 13,
              'cop': 14,
              'num': 15,
              'xcomp': 16,
              'advcl': 17,
              'aux': 18,
     

In [9]:
dep_tag_vocab

{'itos': ['<pad>',
  '<unk>',
  'dep',
  'det',
  'amod',
  'prep',
  'advmod',
  'nsubj',
  'pobj',
  'nn',
  'ccomp',
  'mark',
  'dobj',
  'cc',
  'cop',
  'num',
  'xcomp',
  'advcl',
  'aux',
  'rcmod',
  'csubj',
  'discourse',
  'neg',
  'poss',
  'conj',
  'pcomp',
  'tmod',
  'quantmod',
  'possessive',
  'number',
  'prt',
  'acomp',
  'parataxis',
  'npadvmod',
  'partmod',
  'auxpass',
  'expl',
  'nsubjpass',
  'infmod',
  'iobj'],
 'stoi': defaultdict(<function __main__._default_unk_index()>,
             {'<pad>': 0,
              '<unk>': 1,
              'dep': 2,
              'det': 3,
              'amod': 4,
              'prep': 5,
              'advmod': 6,
              'nsubj': 7,
              'pobj': 8,
              'nn': 9,
              'ccomp': 10,
              'mark': 11,
              'dobj': 12,
              'cc': 13,
              'cop': 14,
              'num': 15,
              'xcomp': 16,
              'advcl': 17,
              'aux': 18,
     

## No Pretraining

| Metrics\Datasets      | Twitter | Restaurant | Laptop     | 
| :---        |    :----:   |  :----:|        ---: |
| Precision      | 0.70       | 0.632 | 0.5950  |
| Recall   | 0.60        | 0.582 | 0.6069     |
| F1   | 0.623        | 0.573 | 0.5919     |
| Accuracy   | 0.669        | 0.7366 | 0.6489     |


In [10]:
PATH = "model/rgat_pretrained.pt"
model = TextGraphAttentionNetwork(args, dep_tag_vocab['len']).to(args.device)
model.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [11]:
model

TextGraphAttentionNetwork(
  (embed): Embedding(8191, 300)
  (dropout): Dropout(p=0, inplace=False)
  (tanh): Tanh()
  (gate): Gating(
    (linear): ModuleList(
      (0): Linear(in_features=300, out_features=300, bias=True)
      (1): Linear(in_features=300, out_features=300, bias=True)
    )
    (gate): ModuleList(
      (0): Linear(in_features=300, out_features=300, bias=True)
      (1): Linear(in_features=300, out_features=300, bias=True)
    )
  )
  (rnn): GRU(300, 300, num_layers=2, batch_first=True, bidirectional=True)
  (dep_embed): Embedding(40, 300)
  (fc): Linear(in_features=300, out_features=600, bias=True)
  (fcs): Sequential(
    (0): Linear(in_features=1200, out_features=300, bias=True)
    (1): ReLU()
    (2): Linear(in_features=300, out_features=300, bias=True)
    (3): ReLU()
  )
  (fc_final): Linear(in_features=300, out_features=3, bias=True)
  (linear): Linear(in_features=600, out_features=600, bias=True)
)

In [12]:
# Train
all_eval_results = train(args, train_dataset, model, test_dataset)

best_eval_result = max(all_eval_results, key=lambda x: x['acc']) 
print("Best Eval result is: ")
print(best_eval_result)

Started Training...
Num examples = 2313
Num Epochs = 30
Instantaneous batch size per GPU = 16


D:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch: 1; train_loss: 1.052; eval_loss: 0.9393; eval_acc: 0.5204; eval_f1: 0.3487
Epoch: 2; train_loss: 0.8297; eval_loss: 0.7798; eval_acc: 0.663; eval_f1: 0.5709
Epoch: 3; train_loss: 0.6906; eval_loss: 0.7486; eval_acc: 0.6693; eval_f1: 0.5626
Epoch: 4; train_loss: 0.5673; eval_loss: 0.8583; eval_acc: 0.6505; eval_f1: 0.547
Epoch: 5; train_loss: 0.4878; eval_loss: 0.9729; eval_acc: 0.6708; eval_f1: 0.5886
Epoch: 6; train_loss: 0.4373; eval_loss: 1.0091; eval_acc: 0.6599; eval_f1: 0.5708
Epoch: 7; train_loss: 0.3824; eval_loss: 1.0011; eval_acc: 0.6661; eval_f1: 0.6106
Epoch: 8; train_loss: 0.3643; eval_loss: 1.0141; eval_acc: 0.6897; eval_f1: 0.6327
Epoch: 9; train_loss: 0.3183; eval_loss: 1.2132; eval_acc: 0.6803; eval_f1: 0.6113
Epoch: 10; train_loss: 0.274; eval_loss: 1.2525; eval_acc: 0.6677; eval_f1: 0.596
Epoch: 11; train_loss: 0.2651; eval_loss: 1.5822; eval_acc: 0.6912; eval_f1: 0.6288
Epoch: 12; train_loss: 0.2833; eval_loss: 1.2333; eval_acc: 0.6755; eval_f1: 0.6061
Epoch: